In [1]:
import pandas as pd

In [2]:
def add_week_day(df, date_column="timestamp"):
    df = df.copy()
    df["week_day"] = df[date_column].dt.day_name()
    return df

In [3]:
def calcular_jornada(df, start_event=1, end_event=4):
    jornada = df[df["event_id"].isin([start_event, end_event])].copy()
    jornada = jornada.sort_values("timestamp").reset_index(drop=True)

    # Cada evento de inicio inicia un bloque
    jornada["bloque"] = (jornada["event_id"] == start_event).cumsum()

    # Pivot para tener inicio y fin
    bloques = jornada.pivot_table(
        index=["bloque", jornada["timestamp"].dt.date, "week_day"],
        columns="event_id",
        values="timestamp",
        aggfunc="first"
    )

    bloques = bloques.rename(columns={start_event: "start", end_event: "end"})

    # Solo bloques completos
    bloques = bloques.dropna(subset=["start", "end"])

    # Duración del bloque
    bloques["duration"] = bloques["end"] - bloques["start"]

    # Reset index y renombrar columna de fecha
    bloques = bloques.reset_index()
    bloques = bloques.rename(columns={"timestamp": "date"})

    return bloques[["date", "week_day", "start", "end", "duration"]]

In [4]:
def calcular_concentracion(df, start_event=2, end_event=3):
    trabajo = df[df["event_id"].isin([start_event, end_event])].copy()
    trabajo = trabajo.sort_values("timestamp").reset_index(drop=True)

    # Cada evento 2 inicia un bloque
    trabajo["block"] = (trabajo["event_id"] == start_event).cumsum()

    # Pivot para tener inicio y fin
    blocks = trabajo.pivot_table(
        index=["block", trabajo["timestamp"].dt.date, "week_day"],
        columns="event_id",
        values="timestamp",
        aggfunc="first"
    )

    blocks = blocks.rename(columns={start_event: "start", end_event: "end"})

    # Solo bloques completos
    blocks = blocks.dropna(subset=["start", "end"])

    # Duración del bloque
    blocks["duration"] = blocks["end"] - blocks["start"]

    blocks = blocks.reset_index()
    blocks = blocks.rename(columns={"timestamp": "date"})

    return blocks[["date", "week_day", "start", "end", "duration"]]

In [5]:
def daily_summary(jornada_df, concentracion_df):
    jornada_sum = jornada_df.groupby("date")["duration"].sum().rename("work_time")
    concentracion_sum = concentracion_df.groupby("date")["duration"].sum().rename("focus_time")
    non_concentration_sum = (jornada_sum - concentracion_sum).rename("non_focus_time")
    concentracion_count = concentracion_df.groupby("date").size().rename("concentration_blocks")

    resumen = pd.concat([jornada_sum, concentracion_sum, non_concentration_sum, concentracion_count], axis=1).reset_index()

    #resumen["non_focus_time"] = resumen["work_time"] - resumen["focus_time"]

    # Opcional: horas en decimal
    resumen["work_hours"] = resumen["work_time"].dt.total_seconds() / 3600
    resumen["focus_hours"] = resumen["focus_time"].dt.total_seconds() / 3600
    resumen["non_focus_hours"] = resumen["work_hours"] - resumen["focus_hours"]

    resumen["focus_percentage"] = (resumen["focus_time"] / resumen["work_time"]) * 100
    resumen["non_focus_percentage"] = (resumen["non_focus_time"] / resumen["work_time"]) * 100

    resumen["avg_block_time"] = resumen["focus_time"] / resumen["concentration_blocks"]
    resumen["avg_block_hours"] = resumen["focus_hours"] / resumen["concentration_blocks"]

    return resumen

In [6]:
# Cargar CSV
df = pd.read_csv("concentration_tracking.csv")

In [7]:
df

,date,time,event_id
0,1/9/2026,9:36:29,1
1,1/9/2026,9:36:33,2
2,1/9/2026,9:43:53,3
3,1/9/2026,10:00:57,2
4,1/9/2026,10:39:52,3
...,...,...,...
175,2026-01-23,15:18:41,2
176,2026-01-23,17:47:35,3
177,2026-01-23,17:57:47,2
178,2026-01-23,19:01:45,3


In [8]:
#consolidate date format
df["date"] = pd.to_datetime(df["date"].astype(str), format="mixed")

#combine date and time in a single column
df["timestamp"] = pd.to_datetime(
    df["date"].astype(str) + " " + df["time"].astype(str),
    format="mixed"
)

#order by timestamp
df = df.sort_values("timestamp").reset_index(drop=True)

In [9]:
df = add_week_day(df)

#dataframe overview
df

,date,time,event_id,timestamp,week_day
0,2026-01-09,9:36:29,1,2026-01-09 09:36:29,Friday
1,2026-01-09,9:36:33,2,2026-01-09 09:36:33,Friday
2,2026-01-09,9:43:53,3,2026-01-09 09:43:53,Friday
3,2026-01-09,10:00:57,2,2026-01-09 10:00:57,Friday
4,2026-01-09,10:39:52,3,2026-01-09 10:39:52,Friday
...,...,...,...,...,...
175,2026-01-23,15:18:41,2,2026-01-23 15:18:41,Friday
176,2026-01-23,17:47:35,3,2026-01-23 17:47:35,Friday
177,2026-01-23,17:57:47,2,2026-01-23 17:57:47,Friday
178,2026-01-23,19:01:45,3,2026-01-23 19:01:45,Friday


In [13]:
df_j = calcular_jornada(df)
df_j

event_id,date,week_day,start,end,duration
0,2026-01-09,Friday,2026-01-09 09:36:29,2026-01-09 12:34:10,0 days 02:57:41
1,2026-01-09,Friday,2026-01-09 15:04:43,2026-01-09 16:10:27,0 days 01:05:44
2,2026-01-12,Monday,2026-01-12 09:38:29,2026-01-12 14:09:12,0 days 04:30:43
3,2026-01-12,Monday,2026-01-12 15:02:23,2026-01-12 18:14:49,0 days 03:12:26
4,2026-01-13,Tuesday,2026-01-13 09:01:03,2026-01-13 14:25:52,0 days 05:24:49
5,2026-01-13,Tuesday,2026-01-13 15:22:19,2026-01-13 16:49:45,0 days 01:27:26
6,2026-01-14,Wednesday,2026-01-14 09:03:59,2026-01-14 13:47:22,0 days 04:43:23
7,2026-01-14,Wednesday,2026-01-14 15:11:08,2026-01-14 16:58:29,0 days 01:47:21
8,2026-01-15,Thursday,2026-01-15 09:01:41,2026-01-15 14:23:30,0 days 05:21:49
9,2026-01-15,Thursday,2026-01-15 15:05:45,2026-01-15 17:01:31,0 days 01:55:46


In [14]:
df_c = calcular_concentracion(df)
df_c

event_id,date,week_day,start,end,duration
0,2026-01-09,Friday,2026-01-09 09:36:33,2026-01-09 09:43:53,0 days 00:07:20
1,2026-01-09,Friday,2026-01-09 10:00:57,2026-01-09 10:39:52,0 days 00:38:55
2,2026-01-09,Friday,2026-01-09 11:00:25,2026-01-09 11:40:43,0 days 00:40:18
3,2026-01-09,Friday,2026-01-09 11:51:58,2026-01-09 12:18:47,0 days 00:26:49
4,2026-01-09,Friday,2026-01-09 15:04:45,2026-01-09 15:17:28,0 days 00:12:43
...,...,...,...,...,...
65,2026-01-23,Friday,2026-01-23 09:33:37,2026-01-23 10:39:57,0 days 01:06:20
66,2026-01-23,Friday,2026-01-23 11:09:13,2026-01-23 14:00:18,0 days 02:51:05
67,2026-01-23,Friday,2026-01-23 14:31:29,2026-01-23 15:14:57,0 days 00:43:28
68,2026-01-23,Friday,2026-01-23 15:18:41,2026-01-23 17:47:35,0 days 02:28:54


In [15]:
r_df = daily_summary(df_j, df_c)
r_df["date"] = pd.to_datetime(r_df["date"].astype(str))

In [16]:
r_df = add_week_day(r_df, "date")
r_df

,date,work_time,focus_time,non_focus_time,concentration_blocks,work_hours,focus_hours,non_focus_hours,focus_percentage,non_focus_percentage,avg_block_time,avg_block_hours,week_day
0,2026-01-09,0 days 04:03:25,0 days 02:37:48,0 days 01:25:37,7,4.056944,2.630000,1.426944,64.827114,35.172886,0 days 00:22:32.571428571,0.375714,Friday
1,2026-01-12,0 days 07:43:09,0 days 05:12:42,0 days 02:30:27,8,7.719167,5.211667,2.507500,67.515924,32.484076,0 days 00:39:05.250000,0.651458,Monday
2,2026-01-13,0 days 06:52:15,0 days 06:00:18,0 days 00:51:57,5,6.870833,6.005000,0.865833,87.398423,12.601577,0 days 01:12:03.600000,1.201000,Tuesday
3,2026-01-14,0 days 06:30:44,0 days 04:34:09,0 days 01:56:35,10,6.512222,4.569167,1.943056,70.162941,29.837059,0 days 00:27:24.900000,0.456917,Wednesday
4,2026-01-15,0 days 07:17:35,0 days 05:59:56,0 days 01:17:39,8,7.293056,5.998889,1.294167,82.254809,17.745191,0 days 00:44:59.500000,0.749861,Thursday
5,2026-01-19,0 days 07:52:12,0 days 04:44:15,0 days 03:07:57,11,7.870000,4.737500,3.132500,60.196950,39.803050,0 days 00:25:50.454545454,0.430682,Monday
6,2026-01-20,0 days 04:28:06,0 days 02:52:42,0 days 01:35:24,4,4.468333,2.878333,1.590000,64.416263,35.583737,0 days 00:43:10.500000,0.719583,Tuesday
7,2026-01-21,0 days 07:23:56,0 days 04:14:22,0 days 03:09:34,7,7.398889,4.239444,3.159444,57.298393,42.701607,0 days 00:36:20.285714285,0.605635,Wednesday
8,2026-01-22,0 days 05:41:56,0 days 02:19:56,0 days 03:22:00,5,5.698889,2.332222,3.366667,40.924157,59.075843,0 days 00:27:59.200000,0.466444,Thursday
9,2026-01-23,0 days 08:57:12,0 days 08:13:45,0 days 00:43:27,5,8.953333,8.229167,0.724167,91.911765,8.088235,0 days 01:38:45,1.645833,Friday
